In [2]:
import speech_recognition as sr
import pyttsx3
import datetime
import requests
from bs4 import BeautifulSoup

# Initialize recognizer and TTS engine
recognizer = sr.Recognizer()
tts_engine = pyttsx3.init()

def speak(text):
    tts_engine.say(text)
    tts_engine.runAndWait()

def listen():
    with sr.Microphone() as source:
        print("Listening...")
        audio = None
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
        except sr.WaitTimeoutError:
            speak("I didn't hear anything. Please try again.")
            return None

        try:
            command = recognizer.recognize_google(audio)
            print(f"User said: {command}")
            return command.lower()
        except sr.UnknownValueError:
            speak("Sorry, I didn't catch that. Please repeat.")
            return None

def fetch_search_results(query):
    url = f"https://www.google.com/search?q={query}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    results = []
    for g in soup.find_all('div', class_='BNeawe vvjwJb AP7Wnd'):
        results.append(g.get_text())
    return results[:5]  # Return top 5 results

def respond_to_command(command):
    if "hello" in command:
        speak("Hello! How can I assist you today?")
    elif "time" in command:
        current_time = datetime.datetime.now().strftime("%H:%M")
        speak(f"The current time is {current_time}")
    elif "date" in command:
        current_date = datetime.datetime.now().strftime("%Y-%m-%d")
        speak(f"Today's date is {current_date}")
    elif "search" in command:
        search_query = command.replace("search", "").strip()
        if search_query:
            speak(f"Searching for {search_query}")
            results = fetch_search_results(search_query)
            if results:
                for result in results:
                    speak(result)
            else:
                speak("I couldn't find any results.")
        else:
            speak("What would you like me to search for?")
    elif "stop" in command:
        speak("Stopping the assistant. Goodbye!")
        return False  # Signal to stop the loop
    else:
        speak("I didn't understand that command.")
    return True  # Continue the loop

if __name__ == "__main__":
    while True:
        command = listen()
        if command:
            if not respond_to_command(command):
                break


Listening...
User said: hello
Listening...
Listening...
User said: stop
